In [ ]:
import pandas as pd
data = pd.read_csv('/content/train.tsv', sep = '\t',header= None)
data.columns = ['words','segmentation']
data 

,words,segmentation
0,konsiltan,k-on-s-i-l-t-an
1,depotwa,d-e-p-o-t-w-a
2,sosyopwofesyonèl,s-o-s-y-o-p-w-o-f-e-s-y-o-n-è-l
3,vejetal,v-e-j-e-t-a-l
4,repibliye,r-e-p-i-b-l-i-y-e
...,...,...
12807,Remi,R-e-m-i
12808,diskriminatwa,d-i-s-k-r-i-m-i-n-a-t-w-a
12809,rejè,r-e-j-è
12810,manzè,m-an-z-è


In [ ]:
#data['segmentation'] = data['segmentation'].str.replace('-','  ')
#data['segmentation'] = data['segmentation'].str.lower()

In [ ]:
data_list_division = data['segmentation'].to_list()
res = []
for word_division in data_list_division:
  split_word = word_division.split('-')
  temp = []
  for letter in split_word:
    temp.append((letter[0], 'B'))
    if len(letter) > 1:
      for item in letter[1:]:
        temp.append((item, 'I'))
  res.append(temp)

In [ ]:
vocab = list(set([w for sent in res for (w,t) in sent]))
vocab.append('<PAD>')
print(len(vocab))
tags = list(set([t for sent in res for (w,t) in sent]))
tags.append('<PAD>')
print(tags)
print(res[0]) 

52
['I', 'B', '<PAD>']
[('k', 'B'), ('o', 'B'), ('n', 'I'), ('s', 'B'), ('i', 'B'), ('l', 'B'), ('t', 'B'), ('a', 'B'), ('n', 'I')]


In [ ]:
max_len = 0
for i in res:
  if max_len < len(i):
      max_len = len(i)

max_len

18

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np

In [ ]:
word2index = {w: i for i, w in enumerate(vocab)}
tag2index = {t: i for i, t in enumerate(tags)}
onehot = [[word2index[w[0]] for w in s] for s in res]
x = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(vocab)-1)

In [ ]:
word2index

{'a': 0,
 'e': 1,
 'z': 2,
 'C': 3,
 'è': 4,
 't': 5,
 'k': 6,
 'Z': 7,
 'E': 8,
 'g': 9,
 'y': 10,
 'F': 11,
 'D': 12,
 'S': 13,
 'G': 14,
 'V': 15,
 'à': 16,
 'B': 17,
 'J': 18,
 'I': 19,
 'n': 20,
 'd': 21,
 'Ò': 22,
 'Y': 23,
 'c': 24,
 'O': 25,
 'L': 26,
 'i': 27,
 'w': 28,
 'j': 29,
 'v': 30,
 'T': 31,
 'N': 32,
 'u': 33,
 'p': 34,
 's': 35,
 'h': 36,
 'R': 37,
 'È': 38,
 'r': 39,
 'l': 40,
 'M': 41,
 'm': 42,
 'K': 43,
 'W': 44,
 'A': 45,
 'ò': 46,
 'P': 47,
 'o': 48,
 'b': 49,
 'f': 50,
 '<PAD>': 51}

In [ ]:
onehot_y = [[tag2index[w[1]] for w in s] for s in res]
y = pad_sequences(maxlen=max_len, sequences=onehot_y, padding="post", value=tag2index["<PAD>"])
y = [to_categorical(i, num_classes=len(tags)) for i in y]
y = np.asarray(y)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1, random_state=41)

In [ ]:
#LSTM model using keras library

model = Sequential()
model.add(tf.keras.Input(shape=(x_train.shape[-1]), name="word_input"))
model.add(Embedding(input_dim = len(vocab), output_dim=10,input_length=max_len))

model.add((LSTM(256, return_sequences=True)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.TimeDistributed(Dense(len(tags),activation='softmax')))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 10)            520       
                                                                 
 lstm (LSTM)                 (None, 18, 256)           273408    
                                                                 
 dense (Dense)               (None, 18, 64)            16448     
                                                                 
 dropout (Dropout)           (None, 18, 64)            0         
                                                                 
 time_distributed (TimeDistr  (None, 18, 3)            195       
 ibuted)                                                         
                                                                 
Total params: 290,571
Trainable params: 290,571
Non-trainable params: 0
__________________________________________________

In [ ]:
history = model.fit(x_train, y_train, batch_size=16, epochs=2, validation_split=0.1)

Epoch 1/2
649/649 [==============================] - 45s 62ms/step - loss: 0.1106 - accuracy: 0.9639 - val_loss: 0.0167 - val_accuracy: 0.9953
Epoch 2/2
649/649 [==============================] - 42s 65ms/step - loss: 0.0154 - accuracy: 0.9955 - val_loss: 0.0144 - val_accuracy: 0.9953


In [ ]:
model.evaluate(x_test,y_test)

41/41 [==============================] - 2s 38ms/step - loss: 0.0135 - accuracy: 0.9960


[0.013478398323059082, 0.995969831943512]

In [ ]:
y_true, y_pred = [], []
for i in range(len(x_test)):
  pred = model.predict(np.array([x_test[i]]))
  p = np.argmax(pred, axis=-1)
  p_actual = np.argmax(y_test[i],axis=-1)
  for i in range(len(p_actual)):
    if p_actual[i] == 2: break
    y_true.append(p_actual[i])
    y_pred.append(p[0][i])

In [ ]:
pred = model.predict(np.array([x_test[5]]))
p = np.argmax(pred,axis=-1)
p_actual = np.argmax(y_test[5],axis=-1)
print(p)
print(p_actual)

[[1 0 1 1 1 1 0 1 2 2 2 2 2 2 2 2 2 2]]
[1 0 1 1 1 1 0 1 2 2 2 2 2 2 2 2 2 2]


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true,y_pred)

0.9893324156916724

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_true, y_pred, average=None, labels=[0, 1, 2])

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(array([0.90568182, 0.99872416, 0.        ]),
 array([0.98760843, 0.98950828, 0.        ]),
 array([0.94487255, 0.99409486, 0.        ]),
 array([ 807, 7911,    0]))

In [ ]:
def data_preprocess_pipeline(X, all_words):
  onehot = [[word2index[w[0]] for w in s] for s in all_words]
  X_pad = pad_sequences(maxlen=max_len, sequences=onehot, padding="post", value=len(vocab)-1)
  return X_pad

In [ ]:
def predict_tags(word):
  pred = model.predict(np.array([word]))
  p = np.argmax(pred, axis=-1)
  return p[0]

In [ ]:
def get_graphene(word, processed_word):
  b_i_tags = predict_tags(processed_word)
  ans = [word[0]]
  for i in range(1, len(word)):
    if b_i_tags[i] == 1: ans.append('-')
    ans.append(word[i])
  return "".join(ans)

In [ ]:
def get_all_graphenes(file):
  test_data = []
  all_graphenes = []
  words = []
  all_letters = []
  test_file = open(file, 'r')
  for line in test_file:
    word = line.rstrip("\n")
    test_data.append(word)
    words.append(word)
    temp = []
    for letter in word:
      temp.append(letter)
    all_letters.append(temp)
  test_data = data_preprocess_pipeline(test_data, all_letters)
  for i in range(len(test_data)):
    all_graphenes.append([words[i], get_graphene(words[i], test_data[i])])
  return all_graphenes

In [ ]:
ans = get_all_graphenes('/content/test.txt')

In [ ]:
import csv

with open('super_BiLSTM.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for graphene in ans:
      tsv_writer.writerow(graphene)

In [ ]:
def verify_results(file, num_of_lines):
  test_file = open(file, 'r')
  total = 0
  for line in test_file:
    total += 1
  return total == num_of_lines

In [ ]:
verify_results('super_BiLSTM.tsv', len(ans))

True